In [69]:
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None

In [70]:
def check_dupl_inn(value):
    """
    Функция для добавления нуля если количество цифр равно 9
    """
    if len(value) == 9:
        return f'0{value}'
    else:
        return value
    

In [71]:
def check_bur_inn(value:str):
    """
    Функция для проверки начинается ли инн с 03
    """
    if len(value) == 10 and value.startswith('03'):
        return f'бурятский ИНН'
    else:
        return 'неопределенный ИНН'

In [72]:
all_df = pd.read_excel('data/data.xlsx',dtype=str)

In [73]:
region = 'Республика Бурятия'
sferum_student_threshold = 33 # порог для школьников в сферуме
sferum_teacher_threshold = 51 # порог для учителей в сферуме
myschool_student_threshold = 33 # порог для школьников в моей школе
myschool_student_threshold = 33 # порог для учителей в моей школе


In [74]:
all_df.head(1)

,id,Субъект РФ,Наименование АТО,Краткое наименование организации,Полное наименование организации,Адрес организации,ИНН,Тип поселения,Статус организации\n(юр.лицо/филиал),Статус\nфункционирования,...,"Педагоги, использовавшие за последние 12 месяцев Сферум % от общего числа","Обучающиеся, использовавшие за последние 12 месяцев МШ % от общего числа","Педагоги, использовавшие за последние 12 месяцев МШ % от общего числа","Обучающиеся, использовавшие за последние 12 месяцев Сферум\nБолее 50% - 1\nМенее 50% - 0","Педагоги, использовавшие за последние 12 месяцев Сферум\nБолее 50% - 1\nМенее 50% - 0","Педагоги, использовавшие за последние 12 месяцев Сферум + МШ \nБолее 50% - 1\nМенее 50% - 0","Обучающиеся, использовавшие за последние 12 месяцев Сферум + МШ \nБолее 50% - 1\nМенее 50% - 0","Обучающиеся, использовавшие за последние 12 месяцев Сферму + МШ и Педагоги Сферум + МШ\nБолее 50% - 1\nМенее 50% - 0","Обучающиеся, использовавшие за последние 12 месяцев МШ\nБолее 50% - 1\nМенее 50% - 0","Педагоги, использовавшие за последние 12 месяцев МШ\nБолее 50% - 1\nМенее 50% - 0"
0,16675,Алтайский край,Алейский,"МБОУ ""Большепанюшевская СОШ""","МБОУ ""Большепанюшевская СОШ"" Алейского района,...","658101 Алтайский край, Алейский район, село Бо...",2231003612,сельская местность,юридическое лицо,действующая,...,0.9130434782608695,0.03333333333333333,1.0869565217391304,0,1,1,0,0,0,1


In [75]:
all_df.columns

Index(['id', 'Субъект РФ', 'Наименование АТО',
       'Краткое наименование организации', 'Полное наименование организации',
       'Адрес организации', 'ИНН', 'Тип поселения',
       'Статус организации\n(юр.лицо/филиал)', 'Статус\nфункционирования',
       'Государственная/негосударственная организация (ГОУ/НОУ)',
       'Численность обучающихся\nвсего, чел. (Раздел 1.3. стр.01 гр.3)',
       'Численность педработников - всего, чел. (Раздел 3.1 стр.06 гр.3)',
       'Обучающиеся Сферум (чел)', 'Обучающиеся МШ (чел)',
       'Обучающиеся Cферум или МШ (чел)',
       'Обучающиеся, использовавшие за последние 12 месяцев Сферум',
       ' Педагоги, использовавшие за последние 12 месяцев Сферум',
       'Обучающиеся, использовавшие за последние 12 месяцев МШ',
       ' Педагоги, использовавшие за последние 12 месяцев МШ',
       'Педагоги, использовавшие за последние 12 месяцев Сферум или МШ (чел)',
       'Обучающиеся, использовавшие за последние 12 месяцев Сферум % от общего числа',
   

In [76]:
all_df.shape

(38903, 32)

In [77]:
#Фильруем датафрейм

df = all_df[all_df['Субъект РФ'] == region]

In [78]:
df.shape

(464, 32)

In [79]:
df['ИНН'] = df['ИНН'].fillna('пустой') # заполняем наны 
df['ИНН'] = df['ИНН'].astype(str) # заполняем наны 

In [80]:
df['ИНН']

20549     319002490
20550     323055021
20551     302101485
20552     314004542
20553    0307008401
            ...    
21008    0323091735
21009    0323092048
21010    0323092810
21011    0323098554
21012    0323092584
Name: ИНН, Length: 464, dtype: object

In [81]:
# Обрабатываем ИНН, должно быть 10 цифр
df.insert(7,'ИНН количество цифр',df['ИНН'].apply(lambda x: len(x)))

In [82]:
# Проверям начинается ли ИНН с 03
df.insert(8,'Бурятский ИНН',df['ИНН'].apply(check_bur_inn))

In [83]:
# Добавляем нули для поиска дубликатов
df.insert(9,'Для проверки дубликатов',df['ИНН'].apply(check_dupl_inn))

In [84]:
df.to_excel('data/itog.xlsx',index=False)

In [85]:
'Lindy Booth'

'Lindy Booth'